In [1]:
from itertools import product
from math import sin, cos
from tqdm import tqdm
import numpy as np
from z3 import *

### Constantes globais

In [29]:
V_LOW, V_HIGH = 1, 10 # velocidades dos modos baixo e alto, em m
THETA = 15 # angulo de viragem entre modos
TAU = 1 # tempo minimo entre transicoes timed
ALL_ROUTES = [i*THETA for i in range(int(360/THETA))] # todos os ângulos possíveis

GAUSS_DP = 80 # desvio padrão da gaussiana de inicialização das posições em m
SAFE_DIST = 50 # distância de segurança entre barcos em m

### Funções auxiliares

In [3]:
deg_to_rad = lambda a: a * np.pi / 180
rad_to_deg = lambda a: a * 180 / np.pi
z3tofloat = lambda v: float(v.numerator_as_long())/float(v.denominator_as_long())
Abs = lambda x: If(x>=0, x, -x)
val_angle = lambda a: If(a<0, 360+a, If(a>=360, a-360, a))

## Funções de declaração, inicialização e transição

In [30]:
def declare(t, num_boats=3):
    # Iterar os barcos
    trace = {}
    for i in range(num_boats):
        trace[i] = {}
        
        # Variáveis de modo
        trace[i]["v"] = Real(f"b{i}_v{t}")
        trace[i]["a"] = Int(f"b{i}_a{t}")
        
        # Variáveis de estado
        trace[i]["x"] = Real(f"b{i}_x{t}")
        trace[i]["y"] = Real(f"b{i}_y{t}")
        trace[i]["t"] = Real(f"b{i}_t{t}")
        
    return trace
        
        
def init(tr):
    # Gerar ângulos iniciais aleatórios
    random_angles = ALL_ROUTES
    
    # Verificar que há mais barcos que ângulos possíveis
    assert len(tr) <= len(random_angles)
    
    # Iterar os barcos
    r = []
    for i in range(len(tr)):
        np.random.shuffle(random_angles)
        
        # Condições do modo
        r.append(tr[i]["v"] == V_HIGH)
        r.append(tr[i]["a"] == random_angles[0])
        
        # Condições do estado
        r.append(tr[i]["x"] == GAUSS_DP * np.random.randn())
        r.append(tr[i]["y"] == GAUSS_DP * np.random.randn())
        r.append(tr[i]["t"] == 0)
        
    r = And(r)
    
    return r


def switch_safe(tr, boat_id):
    r = []
    
    # Iterar cada um dos barcos
    for i in range(len(tr)):
        if i != boat_id:
            # Condição das distâncias
            x = Abs(tr[i]["x"]-tr[boat_id]["x"]) > SAFE_DIST
            y = Abs(tr[i]["y"]-tr[boat_id]["y"]) > SAFE_DIST
            
            # Condição do tempo
            v_med = (tr[i]["v"] + tr[boat_id]["v"]) / 2
            t = Abs(tr[i]["t"]-tr[boat_id]["t"]) * v_med > SAFE_DIST
            
            # Verificar se uma das distâncias é superior à distância de segurança
            r.append(Or(x, y, t))
            
    r = And(r)
            
    return r
        
    
def timed(prev, curr):
    routes = ALL_ROUTES
    
    # Iterar cada um dos barcos
    r = []
    for i in range(len(prev)):
        
        # Condição dos modos
        r.append(curr[i]["v"] == prev[i]["v"])
        
        # Condição do tempo
        r.append(curr[i]["t"] - prev[i]["t"] > TAU)
        
        # Condição da rota e posição
        routes_cond = []
        for j in range(len(routes)):
            route_cond = []
            
            # Condição do ângulo
            route = routes[j]
            route_cond.append(prev[i]["a"] == route)
            route_cond.append(curr[i]["a"] == route)
            
            # Incremento de posição
            dx = prev[i]["v"] * cos(deg_to_rad(route)) * (curr[i]["t"] - prev[i]["t"])
            dy = prev[i]["v"] * sin(deg_to_rad(route)) * (curr[i]["t"] - prev[i]["t"])
            
            # Condição da posição
            route_cond.append(curr[i]["x"] == prev[i]["x"] + dx)
            route_cond.append(curr[i]["y"] == prev[i]["y"] + dy)
            
            # print(cos(deg_to_rad(route)), sin(deg_to_rad(route)))
            # print(curr[i]["x"] == prev[i]["x"] + dx)
            # print(curr[i]["y"] == prev[i]["y"] + dx)
            # print("")
            
            # Fazer o And de todas as condições
            routes_cond.append(And(route_cond))
        
        # Adicionar uma de todas as rotas possíveis
        r.append(Or(routes_cond))
        
    r = And(r)
        
    return r


def untimed(prev, curr):
    r = []
    
    # Iterar cada um dos barcos
    for i in range(len(prev)):
        
        # Condições da posição e tempo
        r.append(curr[i]["x"] == prev[i]["x"])
        r.append(curr[i]["y"] == prev[i]["y"])
        r.append(curr[i]["t"] == prev[i]["t"])

        # barco V_LOW transita para V_LOW
        low_low = []
        low_low.append(And(prev[i]["v"] == V_LOW, curr[i]["v"] == V_LOW))
        low_low.append(Not(switch_safe(prev, i)))
        low_low.append(Or(curr[i]["a"] == val_angle(prev[i]["a"]+THETA), curr[i]["a"] == val_angle(prev[i]["a"]-THETA)))
        low_low = And(low_low)

        # barco V_LOW transita para barco V_HIGH
        low_high = []
        low_high.append(And(prev[i]["v"] == V_LOW, curr[i]["v"] == V_HIGH))
        low_high.append(switch_safe(prev, i))
        low_high.append(curr[i]["a"] == prev[i]["a"])
        low_high = And(low_high)

        # barco V_HIGH transita para barco V_LOW
        high_low = []
        high_low.append(And(prev[i]["v"] == V_HIGH, curr[i]["v"] == V_LOW))
        high_low.append(Not(switch_safe(prev, i)))
        high_low.append(Or(curr[i]["a"] == val_angle(prev[i]["a"]+THETA), curr[i]["a"] == val_angle(prev[i]["a"]-THETA)))
        high_low = And(high_low)

        # barco V_HIGH transita para barco V_HIGH
        high_high = []
        high_high.append(And(prev[i]["v"] == V_HIGH, curr[i]["v"] == V_HIGH))
        high_high.append(switch_safe(prev, i))
        high_high.append(curr[i]["a"] == prev[i]["a"])
        high_high = And(high_high)

        # Adicionar uma destas possíveis transições
        r.append(Or(low_low, low_high, high_low, high_high))
        
    # O estado não pode ficar igual
    same = []
    for i in range(len(prev)):
        same.append(prev[i]["v"] == curr[i]["v"])
        same.append(prev[i]["a"] == curr[i]["a"])
        same.append(prev[i]["x"] == curr[i]["x"])
        same.append(prev[i]["y"] == curr[i]["y"])
        # same.append(prev[i]["t"] == curr[i]["t"])
    same = Not(And(same))

    # Todas as condições da transição devem ser cumpridas
    r = And(And(r), same)
    
    return r


def trans(prev, curr):
    # Condições timed e untimed
    untimed_cond = untimed(prev, curr)
    timed_cond = timed(prev, curr)
    
    # Condições de sincronismo
    eq_cond = And([curr[i]["t"] == curr[i+1]["t"] for i in range(len(curr)-1)])
    
    r = And(Or(untimed_cond, timed_cond), eq_cond)
    
    return r

In [31]:
def gen_trace(declare, init, trans, k):
    solver = Solver()
    trace = {i: declare(i) for i in range(k)}
    solver.add(init(trace[0]))
    
    for i in range(k-1):
        solver.add(trans(trace[i], trace[i+1]))
        
    if solver.check() == sat:
        m = solver.model()
        
        for i in range(k):
            print(f"\nTrace {i}:")
            for b in range(len(trace[i])):
                for v in trace[i][b]:
                    if trace[i][b][v].sort() == RealSort():
                        print(f"  {v}{b} = {z3tofloat(m[trace[i][b][v]])}")
                    else:
                        print(f"  {v}{b} = {m[trace[i][b][v]]}")
                print("")
            print("")
                
        r = m
    else:
        r = None
        
    return r

m = gen_trace(declare, init, trans, 10)


Trace 0:
  v0 = 10.0
  a0 = 195
  x0 = 27.48014516721384
  y0 = 116.07568605607688
  t0 = 0.0

  v1 = 10.0
  a1 = 120
  x1 = -43.77045532545815
  y1 = 165.01310277048535
  t1 = 0.0

  v2 = 10.0
  a2 = 300
  x2 = 148.94363958309003
  y2 = -77.44321765008543
  t2 = 0.0



Trace 1:
  v0 = 10.0
  a0 = 195
  x0 = 17.770773532236902
  y0 = 113.47406776747118
  t0 = 1.0051881180440927

  v1 = 10.0
  a1 = 120
  x1 = -48.79639591567861
  y1 = 173.7182872285699
  t1 = 1.0051881180440927

  v2 = 10.0
  a2 = 300
  x2 = 153.96958017331048
  y2 = -86.14840210816998
  t2 = 1.0051881180440927



Trace 2:
  v0 = 10.0
  a0 = 195
  x0 = 8.061401897259962
  y0 = 110.87244947886545
  t0 = 2.0103762360881854

  v1 = 10.0
  a1 = 120
  x1 = -53.82233650589907
  y1 = 182.42347168665444
  t1 = 2.0103762360881854

  v2 = 10.0
  a2 = 300
  x2 = 158.99552076353098
  y2 = -94.85358656625455
  t2 = 2.0103762360881854



Trace 3:
  v0 = 10.0
  a0 = 195
  x0 = -1.6479697377169775
  y0 = 108.27083119025974
  t0 = 3.01